In [ ]:
import os

from tradier_python import TradierAPI

# config
TRADIER_TOKEN = ''
TRADIER_ACCOUNT_ID = ''

ticker = ''

token = TRADIER_TOKEN
account_id = TRADIER_ACCOUNT_ID
t = TradierAPI(token=token, default_account_id=account_id)

profile = t.get_profile()
profile